<a href="https://colab.research.google.com/github/benjaminnigjeh/keyProteoforms/blob/main/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import matplotlib.patches as patches
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from IPython.display import Image

# --- 1. Create Dataset ---
X, y = make_classification(n_samples=1000, n_features=1000, n_classes=2, random_state=42)
X = StandardScaler().fit_transform(X)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# --- 2. Define Model ---
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1000, 64)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.act2 = nn.ReLU()
        self.out = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        z1 = self.fc1(x)
        a1 = self.act1(z1)
        z2 = self.fc2(a1)
        a2 = self.act2(z2)
        out = self.sigmoid(self.out(a2))
        return out, a1

# --- 3. Setup for Animation ---
losses, accuracies = [], []
gradient_mags, weight_snapshots = [], []
activation_snapshots, pred_dists = [], []
fold_info = []

fig, axs = plt.subplots(3, 2, figsize=(12, 10), facecolor='#f7f7f7')
plt.tight_layout(pad=3.0)

# --- 4. Cross-validation + Train ---
kf = KFold(n_splits=5, shuffle=True, random_state=42)
splits = list(kf.split(X))
all_fold_data = []

for fold_idx, (train_idx, test_idx) in enumerate(splits):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    model = Net()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.BCELoss()

    fold_losses, fold_accuracies = [], []
    fold_gradient_mags, fold_weights = [], []
    fold_activations, fold_preds = [], []

    for epoch in range(50):
        model.train()
        for _ in range(5):  # Inner steps
            preds, _ = model(X_train)
            loss = loss_fn(preds, y_train)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            pred_test, a1_test = model(X_test)
            acc = ((pred_test > 0.5) == y_test).float().mean().item()
            weight_fc1 = model.fc1.weight.data.cpu().numpy()
            grad_fc1 = model.fc1.weight.grad.data.cpu().numpy()
            a1_sample = a1_test[:100].detach().cpu().numpy()

        fold_losses.append(loss.item())
        fold_accuracies.append(acc)
        fold_gradient_mags.append(np.abs(grad_fc1).mean())
        fold_weights.append(weight_fc1)
        fold_activations.append(a1_sample)
        fold_preds.append(pred_test.detach().cpu().numpy())
        fold_info.append(fold_idx + 1)

    all_fold_data.append({
        'losses': fold_losses,
        'accuracies': fold_accuracies,
        'gradient_mags': fold_gradient_mags,
        'weights': fold_weights,
        'activations': fold_activations,
        'preds': fold_preds
    })

# Flatten data
losses = sum([fd['losses'] for fd in all_fold_data], [])
accuracies = sum([fd['accuracies'] for fd in all_fold_data], [])
gradient_mags = sum([fd['gradient_mags'] for fd in all_fold_data], [])
weight_snapshots = sum([fd['weights'] for fd in all_fold_data], [])
activation_snapshots = sum([fd['activations'] for fd in all_fold_data], [])
pred_dists = sum([fd['preds'] for fd in all_fold_data], [])

# --- 5. Co-centered Fold + Epoch Wheel ---
def draw_fold_and_epoch_wheel(ax, fold_idx, epoch_idx, total_folds=5, total_epochs=50):
    ax.clear()
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.2, 1.2)
    ax.set_aspect('equal')
    ax.axis('off')

    # Draw fold labels around circle
    radius = 1
    for i in range(total_folds):
        angle = 2 * np.pi * (i / total_folds)
        x = radius * np.sin(angle)
        y = radius * np.cos(angle)
        color = 'red' if i == fold_idx else 'gray'
        size = 14 if i == fold_idx else 10
        ax.text(x, y, str(i + 1), ha='center', va='center', fontsize=size, color=color)

    ax.text(0, 1.1, "Fold", ha='center', fontsize=12)

    # Epoch progress arc
    theta1 = 90
    theta2 = 90 - 360 * (epoch_idx / total_epochs)
    arc = patches.Wedge(center=(0, 0), r=0.7, theta1=theta1, theta2=theta2,
                        width=0.15, facecolor='blue', alpha=0.6)
    ax.add_patch(arc)

    ax.text(0, 0, f"Epoch\n{epoch_idx + 1}", ha='center', va='center', fontsize=12)

# --- 6. Animate ---
def animate(i):
    for ax in axs.flatten():
        ax.clear()

    axs[2, 0].plot(losses[:i+1], label='Loss', color='tomato')
    axs[2, 0].plot(accuracies[:i+1], label='Accuracy', color='teal')
    axs[2, 0].set_title("Loss & Accuracy")
    #axs[0, 0].legend()

    axs[2, 1].hist(pred_dists[i], bins=20, range=(0, 1), color='skyblue', edgecolor='black')
    axs[2, 1].set_title("Prediction Distribution")

    axs[1, 0].imshow(weight_snapshots[i], aspect='auto', cmap='bwr')
    axs[1, 0].set_title("FC1 Weights")

    axs[1, 1].imshow(activation_snapshots[i], aspect='auto', cmap='viridis')
    axs[1, 1].set_title("FC1 Activations")

    axs[0, 1].plot(gradient_mags[:i+1], color='purple')
    axs[0, 1].set_title("Avg. Gradient Magnitude (FC1)")

    draw_fold_and_epoch_wheel(axs[0, 0], fold_info[i] - 1, i % 50)

# --- 7. Save Animation ---
ani = FuncAnimation(fig, animate, frames=len(losses), interval=200)
ani.save("neural_dashboard_kfold_wheel_Final.gif", writer=PillowWriter(fps=5))

# --- 8. Display in Notebook ---
Image(open("neural_dashboard_kfold_wheel_Final.gif", 'rb').read())
